In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import time
import pandas as pd
from urllib3.exceptions import ReadTimeoutError

options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("window-size=1920,1080")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=options)
driver.set_page_load_timeout(10)

tabs_links = []
ingredients_links = []

ingredients_dict = {}
ingredients_dicts_list = []

failed_links = []

try:
    site = "https://incibeauty.com/en/ingredients"
    driver.get(site)

    xpath = "//button[@class='fc-button fc-cta-consent fc-primary-button']"
    WebDriverWait(driver, 10).until(ec.element_to_be_clickable((By.XPATH, xpath))).click()

    xpath = "//ul[@class='list-inline filter-alphanum']//li//a"
    tabs = driver.find_elements(By.XPATH, xpath)
    tabs_links = [element.get_attribute("href") for element in tabs]
    # print(tabs_links)

    for link in tabs_links:
        site = link
        driver.get(site)

        xpath = "//a[@class='color-inherit']"
        WebDriverWait(driver, 10).until(ec.presence_of_all_elements_located((By.XPATH, xpath)))

        ingredients = driver.find_elements(By.XPATH, xpath)
        ingredients_links_list = [element.get_attribute("href") for element in ingredients]
        for ing_link in ingredients_links_list:
            ingredients_links.append(ing_link)

        # print(ingredients_links)

    for index, link in enumerate(ingredients_links):

        if index % 20 == 0 and index != 0:
            driver.quit()
            driver = webdriver.Chrome(options=options)
            driver.set_page_load_timeout(10)

        retry = 3
        while retry > 0:
            try:
                site = link
                driver.get(site)
                time.sleep(2)

                xpath = "//ul[@class='list-unstyled']//li"
                WebDriverWait(driver, 10).until(ec.presence_of_all_elements_located((By.XPATH, xpath)))

                general_info = driver.find_elements(By.XPATH, xpath)
                general_info_values = [element.text for element in general_info]
                # print(general_info_values)

                for i in range(len(general_info_values)):
                    try:
                        key, value = general_info_values[i].split(":", 1)
                        ingredients_dict[key] = value.strip()
                    except ValueError:
                        print(f"Skipping item due to split issue: {general_info_values[i]}")

                xpath = "//span[@class='align-middle']"
                penalty_info = driver.find_elements(By.XPATH, xpath)
                penalty_info_values = [element.text for element in penalty_info]
                key, value = "Penalty", ", ".join(penalty_info_values)
                ingredients_dict[key] = value

                xpath = "//ul[@class='fonctions-inci']//li"
                functions_info = driver.find_elements(By.XPATH, xpath)
                functions_info_values = [element.text for element in functions_info]
                # print(functions_info_values)
                key, value = "Functions", functions_info_values
                ingredients_dict[key] = value
                print(ingredients_dict)

                ingredients_dicts_list.append(ingredients_dict.copy())
                ingredients_dict.clear()

                retry = 0

            except ReadTimeoutError:
                retry -= 1
                print(f"Error loading {site}, {retry} attempts left")
                if retry == 0:
                    print(f"Failed to load {site}")
                    failed_links.append(site)

finally:
    print(":)")
    print("Failed links: ", failed_links)
    driver.quit()

ingredients_df = pd.DataFrame(ingredients_dicts_list)
ingredients_df.to_csv("ingredients_scraped_csv.csv", index=False)
ingredients_df.to_json("ingredients_scraped_json.json", orient="records", indent=4, force_ascii=False)

{'INCI name': '1,2,4-TRIHYDROXYBENZENE', 'EINECS/ELINCS number': '208-575-1', 'Classification': 'Regulated, Hair dye, Forbidden in Europe', 'Penalty': '"Strong penalty" in all categories.', 'Functions': ['Hair dyeing : Colors the hair']}
{'Other language': 'Méthyleugénol', 'INCI name': '1,2-DIMETHOXY-4-(2-PROPENYL)-BENZENE', 'EINECS/ELINCS number': '202-223-0', 'Classification': 'Regulated', 'Penalty': '"Low penalty" in all categories.', 'Functions': []}
{'Other languages': '1,2-Heksandiol, 1,2-Hexaandiol, 1,2-Hexandiol, 1,2-Hexanodiol, 1,2-Hexándiol, 1,2-esandiolo, 1,2-heksandiol, 1,2-heksanodiol, 1,2-hexandiol, 1,2-hexándiol, 1,2-хександиол, 1,2-הקסנדיול, 1,2-ヘキサンジオール, 1,2-헥산디올, 1،2-هيكسانديول', 'INCI name': '1,2-HEXANEDIOL', 'EINECS/ELINCS number': '230-029-6', 'Penalty': '"No penalty" in all categories.', 'Functions': ['Skin conditioning : Maintains skin in good condition', 'Solvent : Dissolves other substances']}
{'INCI name': '1,3-BIS-(2,4-DIAMINOPHENOXY)PROPANE', 'EINECS/ELINCS 

In [1]:
import pandas as pd

ingredients_data = "ingredients_scraped_json.json"
ingredients_df = pd.read_json(ingredients_data)

ingredients_columns = ["INCI name", "Classification", "Penalty", "Functions", "Origin(s)"]
ingredients_df = ingredients_df[ingredients_columns]

ingredients_df = ingredients_df.rename(columns={"Origin(s)": "Origin"})

def to_list(txt):
    if txt is None:
        return []
    else:
        txt_list = [x.strip() for x in txt.split(",")]
    return txt_list

def preprocessing_text(txt):
    if txt is None:
        return "Not available"
    txt = txt.replace("\"", "").replace(".", "").replace(" :,", ":")
    return txt
    
def preprocessing_text_list(txt_list):
    while "" in txt_list:
        txt_list.remove("")
    else:
        for txt_index in range(len(txt_list)):
            txt = txt_list[txt_index]
            txt = txt.replace(".", "").replace(" : ", ": ")
            txt_list[txt_index] = txt 
    return txt_list


def preprocessing_penalty(txt):
    if txt == "Strong penalty in all categories":
        return {"Hair penalty": "Strong", "Body penalty": "Strong", "Face penalty": "Strong",
                "Oral cavity penalty": "Strong", "Makeup penalty": "Strong", "Babies penalty": "Strong"}
    elif txt == "Low penalty in all categories":
        return {"Hair penalty": "Low", "Body penalty": "Low", "Face penalty": "Low", "Oral cavity penalty": "Low",
                "Makeup penalty": "Low", "Babies penalty": "Low"}
    elif txt == "No penalty in all categories":
        return {"Hair penalty": "None", "Body penalty": "None", "Face penalty": "None", "Oral cavity penalty": "None",
                "Makeup penalty": "None", "Babies penalty": "None"}
    elif txt == "Medium penalty in all categories":
        return {"Hair penalty": "Medium", "Body penalty": "Medium", "Face penalty": "Medium",
                "Oral cavity penalty": "Medium", "Makeup penalty": "Medium", "Babies penalty": "Medium"}
    elif txt == "Strong penalty in the following categories: Hair colouring, Low penalty in all other categories":
        return {"Hair penalty": "Strong", "Body penalty": "Low", "Face penalty": "Low", "Oral cavity penalty": "Low",
                "Makeup penalty": "Low", "Babies penalty": "Low"}
    elif txt == ("Strong penalty in the following categories: Breath, Children toothpaste, Adult toothpaste, "
               "Cream for braces, Children toothbrush, Adult toothbrush, Anti-stain and teeth whitening, "
               "Solid toothpaste, Low penalty in all other categories, Penalised in lip products, "
               "Penalised in loose make-up powders, Penalized in spray products"):
        return {"Hair penalty": "Low", "Body penalty": "Low", "Face penalty": "Low", "Oral cavity penalty": "Strong",
                "Makeup penalty": "Strong", "Babies penalty": "Low"}
    elif txt == "Low penalty in all categories, Prohibited in spray cosmetics":
        return {"Hair penalty": "Low", "Body penalty": "Low", "Face penalty": "Low", "Oral cavity penalty": "Low",
                "Makeup penalty": "Low", "Babies penalty": "Low"}
    elif txt == "Medium penalty in all categories, Avoid in children under 3 years old":
        return {"Hair penalty": "Medium", "Body penalty": "Medium", "Face penalty": "Medium",
                "Oral cavity penalty": "Medium", "Makeup penalty": "Medium", "Babies penalty": "Strong"}
    elif txt == ("Strong penalty in the following categories: Baby bubble bath, Wipes box, Baby set, Baby care set, "
               "Diaper cream, Baby cleansing water, Baby shower gel and cream, Baby bath oil, Baby massage oil, "
               "Baby care oil, Nose hygiene, Baby cleansing milk, Baby milk and moisturizer, Baby disinfectant wipes, "
               "Baby cleaning wipes, Liniment, Baby washing foam, Baby solid soap, Physiological serum, Baby shampoo, "
               "2-in-1 shampoo and body wash for baby, Milk crust care, First teeth care, Talc and powder, "
               "Medium penalty in all other categories"):
        return {"Hair penalty": "Medium", "Body penalty": "Medium", "Face penalty": "Medium",
                "Oral cavity penalty": "Medium", "Makeup penalty": "Medium", "Babies penalty": "Strong"}
    elif txt == ("No penalty in the following categories: Baby shower gel and cream, Shower gel, Shaving cream, "
               "Liquid soap, Shaving soap, Low penalty in all other categories"):
        return {"Hair penalty": "Low", "Body penalty": "None", "Face penalty": "Low",
                "Oral cavity penalty": "Low", "Makeup penalty": "Low", "Babies penalty": "Low"}
    elif txt == ("Strong penalty in the following categories: Breath, Children toothpaste, Adult toothpaste, "
               "Cream for braces, Mouthwash, Anti-stain and teeth whitening, Solid toothpaste, Baby bubble bath, "
               "Wipes box, Baby set, Baby care set, Diaper cream, Various baby care, Baby cleansing water, "
               "Baby shower gel and cream, Baby bath oil, Baby massage oil, Baby care oil, Nose hygiene, "
               "Baby cleansing milk, Baby milk and moisturizer, Baby cleaning wipes, Liniment, Baby washing foam, "
               "Baby solid soap, Physiological serum, 2-in-1 shampoo and body wash for baby, Milk crust care, "
               "First teeth care, Talc and powder, Low penalty in all other categories"):
        return {"Hair penalty": "Low", "Body penalty": "Low", "Face penalty": "Low",
                "Oral cavity penalty": "Strong", "Makeup penalty": "Low", "Babies penalty": "Strong"}
    elif txt == ("No penalty in the following categories: Shaving soap, Solid soap, Baby solid soap, Solid shampoo, "
               "Low penalty in all other categories"):
        return {"Hair penalty": "Low", "Body penalty": "None", "Face penalty": "Low",
                "Oral cavity penalty": "Low", "Makeup penalty": "Low", "Babies penalty": "Low"}
    elif txt == ("Strong penalty in the following categories: Diaper cream, Various baby care, Talc and powder, "
               "Sanitary pads, Personal hygiene, Low penalty in all other categories"):
        return {"Hair penalty": "Low", "Body penalty": "Strong", "Face penalty": "Low",
                "Oral cavity penalty": "Low", "Makeup penalty": "Low", "Babies penalty": "Strong"}
    else:
        return {"Hair penalty": "Not available", "Body penalty": "Not available", "Face penalty": "Not available",
                "Oral cavity penalty": "Not available", "Makeup penalty": "Not available", "Babies penalty": "Not available"}

ingredients_df["Classification"] = ingredients_df["Classification"].apply(to_list)
ingredients_df["Origin"] = ingredients_df["Origin"].apply(to_list)
ingredients_df["Penalty"] = ingredients_df["Penalty"].apply(preprocessing_text)
ingredients_df["Functions"] = ingredients_df["Functions"].apply(preprocessing_text_list)

new_penalty_columns = ingredients_df["Penalty"].apply(preprocessing_penalty).apply(pd.Series)
ingredients_df = ingredients_df.join(new_penalty_columns)


ingredients_df = ingredients_df[["INCI name", "Hair penalty", "Body penalty", "Face penalty",
"Oral cavity penalty", "Makeup penalty", "Babies penalty", "Classification", "Functions", "Origin"]]

ingredients_df.to_csv("ingredients_with_penalty_csv.csv", index=False)
ingredients_df.to_json("ingredients_with_penalty_json.json", orient="records", indent=4, force_ascii=False)


In [16]:
import pandas as pd
import re
from difflib import SequenceMatcher
from unidecode import unidecode

cosmetics_data = "cosmetics_original.csv"
cosmetics_df = pd.read_csv(cosmetics_data)

ingredients_data = "ingredients_with_penalty_json.json"
ingredients_df = pd.read_json(ingredients_data)

ingredients_names = ingredients_df["INCI name"]

cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "#NAME?"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Ready Steady Glow Daily AHA Tonic"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Keep Young And Beautiful™ Instant Firming Beauty Shot"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "The Microdelivery Resurfacing Peel"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Multi-Masking™ Medley"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Tight & Bright Clay Multi-Mask"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Tinted Moisturizer SPF 20 - Illuminating"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Cicapair™ Tiger Grass Cream"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Cicapair ™ Tiger Grass Color Correcting Treatment SPF 30"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Powerful-Strength Line-Reducing Concentrate 12.5% Vitamin C"]
cosmetics_df = cosmetics_df[cosmetics_df["Name"] != "Black Tea Kombucha Antioxidant Essence"]

cosmetics_df = cosmetics_df[cosmetics_df["Ingredients"] != "#NAME?"]
cosmetics_df = cosmetics_df[cosmetics_df["Ingredients"] != "No Info"]
cosmetics_df = cosmetics_df[~cosmetics_df['Ingredients'].str.contains("visit", case=False, na=False)]


def preprocessing_ingredients(txt):
    txt = txt.upper()
    if txt[0] == "*" or txt[0] == "-":
        return "Not available"
    else:
        txt = txt.replace("*", "").replace("+", "").replace(";", ",").replace(r"(AND)", ",").replace("-¬‐", "-")
        txt = txt.replace("&", ",").replace("- ", "").replace("--", "-").replace("�", "").replace("PITERA", "")
        txt = txt.replace("HONEY", "MEL").replace("THE CHARCOAL", "CHARCOAL").replace("FRAGRANCE", "PARFUM")
        txt = txt.replace("PLANT ORIGIN", "").replace("FRAGRANCES OF NATURAL ORIGIN", "").replace("NATURAL", "")
        txt = txt.replace("CERTIFIED ORGANIC", "").replace("ORGANIC", "").replace("ESSENTIAL OIL", "")
        txt = re.sub(r"(?:^|,)\s*WATER\s*(?=,|$)", ", AQUA", txt)
        txt = re.sub(r"^,", "", txt)
        txt = re.sub(r"\s?\d+(\.\d+)?%", "", txt)
        txt = re.sub(r"\[.*?]", "", txt)
        txt = re.sub(r" MAY\s?CONTAIN.*", "", txt)
        txt = re.sub(r" RDS PRODUCT NAME.*", "", txt)
        txt = re.sub(r" ORGANIC ESSENTIAL OILS.*", "", txt)
        txt = re.sub(r"^.*?INGREDIENTS: ", "", txt)
        txt = txt.replace("INGREDIENTS: ", ",")
        txt = re.sub(r" PLEASE.*", "", txt)
        txt = re.sub(r" NAPIERS.*", "", txt)
        txt = re.sub(r" MASK.*", "", txt)
        txt = re.sub(r"^.*?:\s*", "", txt)
        txt = re.sub(r"[^a-zA-Z]+$", "", txt)
        txt = re.sub(r"ALCOHOL DENAT(?!\.)", r"ALCOHOL DENAT.", txt)
        txt = txt.replace("OCTOCRYLENEWATER", "OCTOCRYLENE")
    return txt


def eliminating_brackets(txt):
    pattern = r"\([^()]*,[^()]*\)"
    txt = re.sub(pattern, "", txt)
    txt = txt.replace("  ", " ")

    ingredients_list = txt.split(",")
    ingredients_list = [x.strip() for x in ingredients_list]
    for ing in range(len(ingredients_list)):
        ingredient = ingredients_list[ing]
        match = re.search(r"\((.*?)\)", ingredient)
        if ingredient in ingredients_names.values:
            continue
        elif match:
            text_in_brackets = match.group(1)
            if text_in_brackets in ingredients_names.values:
                ingredients_list[ing] = text_in_brackets
            else:
                ingredients_list[ing] = re.sub(r"\s*\([^)]*\)", "", ingredient)
    return ",".join(ingredients_list)


def removing_slashes(txt):
    ingredients_list = txt.split(",")
    ingredients_list = [x.strip() for x in ingredients_list]
    for ing in range(len(ingredients_list)):
        ingredient = ingredients_list[ing]
        if ingredient in ingredients_names.values:
            continue
        elif "/" in ingredient:
            ingredients_slashed = ingredient.split("/")
            ingredients_slashed = [x.strip() for x in ingredients_slashed]
            for slashed_ingredient in ingredients_slashed:
                if slashed_ingredient in ingredients_names.values:
                    ingredients_list[ing] = slashed_ingredient
                    break
                elif slashed_ingredient == ingredients_slashed[-1]:
                    ingredients_list[ing] = ingredients_slashed[0]
                else:
                    continue
    return ingredients_list


def removing_long_strings(txt_list):
    for ingredient in txt_list:
        if len(ingredient) > longest_text:
            too_long.append(txt_list)
            return txt_list
    return txt_list

def removing_short_strings(txt_list):
    new_list = []
    for ingredient in txt_list:
        if len(ingredient) >= shortest_text:
            new_list.append(ingredient)
    return new_list


def finding_similarities(txt_list):
    ingredients_list = txt_list
    protected = ["PEG", "POLYS", "SEED OIL", "PHOSPHATE", "OIL", "HYALURONATE", "FRUIT EXTRACT", "MAGNESIUM",
                 "LEAF EXTRACT", "EXTRACT", "PEG-6", "PEG-4", "PEG15", "MAGNE", "LAURETH", "PCA", "BUTYLPHENYL",
                 "POLYGLYCERYL-6", "PHENOXYETHANO", "ETHANOL", "ETHYLENE", "SELENIUM", "COPPER", "COPOLYMER",
                 "VINYL DIMETHICONE", "PCA DIMETHICONE", "FERMENT", "CASTOR OIL", "ETHYLENEDIAMINE", "LAURETH-2",
                 "LINOLEIC ACID", "AGAR", "BLUE", "ZINC", "RED 4", "BLUE 1"]

    for ing in range(len(ingredients_list)):
        ingredient = ingredients_list[ing]
        if ingredient not in ingredients_names.values:
            if ingredient == "WATER":
                ingredients_list[ing] = "AQUA"
            else:
                matched = False
                for name in ingredients_names.values:
                    if ("/" in name or "-" in name) and ingredient in name and ingredient not in protected:
                        # print(f"matched {ingredient}, {name}")
                        matched = True
                        ingredients_list[ing] = name
                        break
                if not matched:
                    # print(f"not matched {ingredients_list[ing]}")
                    fixed = re.sub(r"(?<=[a-zA-Z])-(?=[a-zA-Z0-9])", "", ingredient)
                    ratings = ingredients_df["INCI name"].apply(
                        lambda x: SequenceMatcher(None, fixed, x).ratio())
                    max_rating = ratings.max()
                    max_rating_index = ratings.idxmax()
                    max_rating_name = ingredients_df.loc[max_rating_index, "INCI name"]

                    if round(max_rating, 2) >= 0.95:
                        # print(f">=0.95 MATCHED: {fixed}, {max_rating_name}, rating: {max_rating}")
                        ingredients_list[ing] = max_rating_name
                        continue
                    else:
                        # print(f"<0.9 NOT MATCHED: {ingredients_list[ing]}, {max_rating_name}")
                        continue

    return ingredients_list

def clean_name(name):
    name = re.sub(r"[™®]", "", name)
    name = unidecode(name)
    
    return name

cosmetics_df["Ingredients"] = cosmetics_df["Ingredients"].apply(preprocessing_ingredients)
cosmetics_df["Ingredients"] = cosmetics_df["Ingredients"].apply(eliminating_brackets)

cosmetics_df = cosmetics_df[cosmetics_df["Ingredients"] != "Not available"]
cosmetics_df = cosmetics_df[~cosmetics_df["Ingredients"].str.contains("INGREDIENT", na=False)]
cosmetics_df = cosmetics_df[~cosmetics_df["Ingredients"].str.contains(":", na=False)]
cosmetics_df = cosmetics_df[~cosmetics_df["Ingredients"].str.contains(r"\(", na=False)]
cosmetics_df = cosmetics_df[~cosmetics_df["Ingredients"].str.contains(r"\)", na=False)]

cosmetics_df["Ingredients"] = cosmetics_df["Ingredients"].apply(removing_slashes)

longest_text = int(ingredients_df["INCI name"].astype(bytes).str.len().max())
too_long = []

shortest_text = int(ingredients_df["INCI name"].astype(bytes).str.len().min())

cosmetics_df["Ingredients"] = cosmetics_df["Ingredients"].apply(removing_long_strings)
cosmetics_df = cosmetics_df[~cosmetics_df["Ingredients"].isin(too_long)]

cosmetics_df["Ingredients"] = cosmetics_df["Ingredients"].apply(removing_short_strings)

cosmetics_df["Ingredients"] = cosmetics_df["Ingredients"].apply(finding_similarities)

cosmetics_df["Name"] = cosmetics_df["Name"].apply(clean_name)
cosmetics_df["Brand"] = cosmetics_df["Brand"].apply(clean_name)

skin_types = ["Combination", "Dry", "Normal", "Oily", "Sensitive"]
cosmetics_df["Skin"] = cosmetics_df.apply(lambda record: [skin for skin in skin_types if record[skin] == 1], axis=1)
cosmetics_df = cosmetics_df.drop(columns=skin_types)

cosmetics_df.to_csv("cosmetics_processed_csv.csv", index=False)
cosmetics_df.to_json("cosmetics_processed_json.json", orient="records", indent=4, force_ascii=False)


In [9]:
from pymongo import MongoClient
import json
import datetime

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_collection = engineering_db.Cosmetics
ingredients_collection = engineering_db.Ingredients

cosmetics_json = "cosmetics_processed_json.json"
ingredients_json = "ingredients_with_penalty_json.json"

with open(cosmetics_json, "r", encoding="utf-8") as file:
    cosmetics = json.load(file)

cosmetics_collection.insert_many(cosmetics)
# cosmetics_collection.update_many({}, {"$set": {"Date": datetime.datetime.now(datetime.timezone.utc)}})
# cosmetics_collection.update_many({}, {"$unset": {"Date": None}})

with open(ingredients_json, "r", encoding="utf-8") as file:
    ingredients = json.load(file)
    
ingredients_collection.insert_many(ingredients)
ingredients_collection.update_many({}, {"$set": {"Date": datetime.datetime.now(datetime.timezone.utc)}})


UpdateResult({'n': 2492, 'nModified': 2492, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [10]:
# old version

ingredients_distinct = cosmetics_collection.distinct("Ingredients")
print(len(ingredients_distinct))

def missing_ingredients(ing_list):
    existing_list = list(ingredients_collection.find({"INCI name": {"$in": ing_list}}))
    existing_names = {ingredient["INCI name"] for ingredient in existing_list}
    nonexistent_names = set(ing_list) - existing_names
    return nonexistent_names

missing_names = missing_ingredients(ingredients_distinct)
for missing_name in missing_names:
    ingredients_collection.insert_one({"INCI name": missing_name, "Hair penalty": "Not available",
                                      "Body penalty": "Not available", "Face penalty": "Not available",
                                      "Oral cavity_penalty": "Not available",
                                      "Makeup penalty": "Not available", "Babies penalty": "Not available",
                                      "Classification": [], "Functions": [], "Origin": [],
                                      "Date": datetime.datetime.now(datetime.timezone.utc)})
    
print(len(missing_names))


3396
2332


In [ ]:
# old version, doesn't work due to removing empty fields in db

from pymongo import MongoClient
import pandas as pd

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_collection = engineering_db.Cosmetics
ingredients_collection = engineering_db.Ingredients

cosmetics_data = list(cosmetics_collection.find())
ingredients_data = list(ingredients_collection.find())

cosmetics_df = pd.DataFrame(cosmetics_data)
ingredients_df = pd.DataFrame(ingredients_data)

# print([x for x in ingredients_collection.distinct("Classification")])

def calculate_rating(c_df, i_df, penalty_category):

    ingredients_ratings = []
    final_ratings = []

    cosmetic_names = c_df["Name"]

    for name in cosmetic_names:

        ingredients_sum = 0
        ingredients_count = 0
        penalty_added = 1
        strong_penalty = ["Suspected endocrine disruptor", "Formaldehyde liberator", "Forbidden in Europe"]
        strong_count = 0

        customer_rank = c_df.loc[c_df["Name"] == name, "Rank"].iloc[0]
        cosmetic_ingredients = c_df.loc[c_df["Name"] == name, "Ingredients"].iloc[0]

        for ing in range(len(cosmetic_ingredients)):
            penalty = i_df.loc[i_df["INCI name"] == cosmetic_ingredients[ing], penalty_category].iloc[0]
            classification = i_df.loc[i_df["INCI name"] == cosmetic_ingredients[ing], "Classification"].iloc[0]
            if penalty == "Strong":
                ingredients_count += 1
                if any(x in classification for x in strong_penalty):
                    penalty_added = 0
                else:
                    if strong_count == 1:
                        ingredients_sum += 1
                        penalty_added *= 0.8
            elif penalty == "Medium":
                ingredients_sum += 5
                ingredients_count += 1
                if penalty_added >= 0.5:
                    penalty_added -= 0.5
                else:
                    penalty_added = 0
            elif penalty == "Low":
                ingredients_sum += 8
                ingredients_count += 1
            elif penalty == "None":
                ingredients_sum += 10
                ingredients_count += 1

        if customer_rank == 0:
            rank_weight = 0
            ingredients_weight = 1
        else:
            customer_rank = (customer_rank - 1) * (10 / 4)
            rank_weight = 0.4
            ingredients_weight = 0.6

        if penalty_added < 0:
            penalty_added = 0

        if ingredients_count != 0:
            ingredients_avg = ingredients_sum / ingredients_count
            ingredients_rating = round(ingredients_avg * penalty_added, 2)
            final_rating = round((ingredients_rating * ingredients_weight) + (customer_rank * rank_weight), 2)
        else:
            ingredients_rating = -10
            final_rating = -10

        ingredients_ratings.append(ingredients_rating)
        final_ratings.append(final_rating)

    return ingredients_ratings, final_ratings

ingredients_ratings_col, final_ratings_col = calculate_rating(cosmetics_df, ingredients_df, "Face penalty")
cosmetics_df["Ingredients_rating"] = ingredients_ratings_col
cosmetics_df["Final_rating"] = final_ratings_col

cosmetics_df.to_csv("cosmetics_rating_csv_old.csv", index=False)

In [16]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
ingredients_collection = engineering_db.Ingredients

fields_to_check = ["Classification", "Functions", "Origin"]

def clean_empty_fields(collection, fields):
    for field in fields:
        collection.update_many(
            {field: {"$eq": []}},
            {"$unset": {field: ""}}
        )

clean_empty_fields(ingredients_collection, fields_to_check)


In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_collection = engineering_db.Cosmetics

fields_to_check = ["Skin"]

def clean_empty_fields(collection, fields):
    for field in fields:
        collection.update_many(
            {field: {"$eq": []}},
            {"$unset": {field: ""}}
        )

clean_empty_fields(cosmetics_collection, fields_to_check)

In [1]:
# db no detail ingredients deleted, adding them in the first place deleted earlier

from pymongo import MongoClient

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
ingredients_collection = engineering_db.Ingredients

ingredients_collection.delete_many({"Hair penalty": "Not available"})


DeleteResult({'n': 2332, 'ok': 1.0}, acknowledged=True)

In [ ]:
from pymongo import MongoClient
import datetime

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_collection = engineering_db.Cosmetics
ingredients_collection = engineering_db.Ingredients
cosmetics_joined_collection = engineering_db.CosmeticsJoined

pipeline = [
    {
        "$unwind": {
            "path": "$Ingredients"
        }
    },
    {
        "$lookup": {
            "from": "Ingredients",
            "localField": "Ingredients",
            "foreignField": "INCI name",
            "as": "IngredientDetails"
        }
    },
    {
        "$unwind": {
            "path": "$IngredientDetails",
            "preserveNullAndEmptyArrays": True
        }
    },
    {
        "$group": {
            "_id": "$_id",
            "Label": { "$first": "$Label" },
            "Brand": { "$first": "$Brand" },
            "Name": { "$first": "$Name" },
            "Price": { "$first": "$Price" },
            "Rank": { "$first": "$Rank" },
            "Ingredients": {
                "$push": {
                    "Ingredient": "$Ingredients",
                    "Details": "$IngredientDetails"
                }
            },
            "Skin": { "$first": "$Skin" },
        }
    },
    {
        "$set": {
            "Date": datetime.datetime.now(datetime.timezone.utc)
        }
    }
]


results = cosmetics_collection.aggregate(pipeline)

cosmetics_joined_collection.insert_many(results)


In [14]:
# db unicode fix, applied earlier

from pymongo import MongoClient
import pandas as pd
import re
from unidecode import unidecode

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_joined_collection = engineering_db.CosmeticsJoined

cosmetics_data = list(cosmetics_joined_collection.find())
cosmetics_df = pd.DataFrame(cosmetics_data)

def clean_name(name):
    name = re.sub(r"[™®]", "", name)
    name = unidecode(name)
    
    return name

cosmetics_df["Name"] = cosmetics_df["Name"].apply(clean_name)

for index, row in cosmetics_df.iterrows():
    cosmetic_id = row["_id"]
    updated_name = row["Name"]

    cosmetics_joined_collection.update_one(
        {"_id": cosmetic_id},
        {"$set": {"Name": updated_name}}
    )

cosmetics_df["Brand"] = cosmetics_df["Brand"].apply(clean_name)

for index, row in cosmetics_df.iterrows():
    cosmetic_id = row["_id"]
    updated_name = row["Brand"]

    cosmetics_joined_collection.update_one(
        {"_id": cosmetic_id},
        {"$set": {"Brand": updated_name}}
    )


In [3]:
# db unicode fix, applied earlier

from pymongo import MongoClient
import pandas as pd
import re
from unidecode import unidecode

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_collection = engineering_db.Cosmetics

cosmetics_data = list(cosmetics_collection.find())
cosmetics_df = pd.DataFrame(cosmetics_data)

def clean_name(name):
    name = re.sub(r"[™®]", "", name)
    name = unidecode(name)
    
    return name

cosmetics_df["Name"] = cosmetics_df["Name"].apply(clean_name)

for index, row in cosmetics_df.iterrows():
    cosmetic_id = row["_id"]
    updated_name = row["Name"]

    cosmetics_collection.update_one(
        {"_id": cosmetic_id},
        {"$set": {"Name": updated_name}}
    )

cosmetics_df["Brand"] = cosmetics_df["Brand"].apply(clean_name)

for index, row in cosmetics_df.iterrows():
    cosmetic_id = row["_id"]
    updated_name = row["Brand"]

    cosmetics_collection.update_one(
        {"_id": cosmetic_id},
        {"$set": {"Brand": updated_name}}
    )


In [13]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_joined_collection = engineering_db.CosmeticsJoined

cosmetics_joined_collection.update_many({}, {"$unset": {"Ingredients.$[].Details._id": ""}})


Liczba dokumentów zmodyfikowanych: 946


In [1]:
# old version

from pymongo import MongoClient
import pandas as pd

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_joined_collection = engineering_db.CosmeticsJoined

cosmetics_data = list(cosmetics_joined_collection.find())
cosmetics_df = pd.DataFrame(cosmetics_data)

def calculate_rating(c_df, penalty_category):
    ingredients_ratings = []
    final_ratings = []

    for _, record in c_df.iterrows():
        ingredients_sum = 0
        ingredients_count = 0
        penalty_added = 1
        strong_penalty = ["Suspected endocrine disruptor", "Formaldehyde liberator", "Forbidden in Europe"]
        strong_count = 0

        customer_rank = record["Rank"]
        cosmetic_ingredients = record["Ingredients"]

        for ingredient in cosmetic_ingredients:
            ingredient_details = ingredient.get("Details", None)

            if ingredient_details:
                penalty = ingredient_details.get(penalty_category, None)
                
                # not needed because if details don't exist it won't get to this part of code
                if penalty is None:
                    continue
                
                classification = ingredient_details.get("Classification", [])

                if penalty == "Strong":
                    ingredients_count += 1
                    if any(x in classification for x in strong_penalty):
                        penalty_added = 0
                    else:
                        if strong_count == 1:
                            ingredients_sum += 1
                            penalty_added *= 0.8
                elif penalty == "Medium":
                    ingredients_sum += 5
                    ingredients_count += 1
                    if penalty_added >= 0.5:
                        penalty_added -= 0.5
                    else:
                        penalty_added = 0
                elif penalty == "Low":
                    ingredients_sum += 8
                    ingredients_count += 1
                elif penalty == "None":
                    ingredients_sum += 10
                    ingredients_count += 1

        if customer_rank == 0:
            rank_weight = 0
            ingredients_weight = 1
        else:
            customer_rank = (customer_rank - 1) * (10 / 4)
            rank_weight = 0.4
            ingredients_weight = 0.6

        if penalty_added < 0:
            penalty_added = 0

        if ingredients_count != 0:
            ingredients_avg = ingredients_sum / ingredients_count
            ingredients_rating = round(ingredients_avg * penalty_added, 2)
            final_rating = round((ingredients_rating * ingredients_weight) + (customer_rank * rank_weight), 2)
        else:
            ingredients_rating = -10
            final_rating = -10

        ingredients_ratings.append(ingredients_rating)
        final_ratings.append(final_rating)

    return ingredients_ratings, final_ratings

ingredients_ratings_col, final_ratings_col = calculate_rating(cosmetics_df, "Face penalty")

cosmetics_df["Ingredients_rating"] = ingredients_ratings_col
cosmetics_df["Final_rating"] = final_ratings_col

cosmetics_df.to_csv("cosmetics_rating_csv.csv", index=False)

In [18]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_joined_collection = engineering_db.CosmeticsJoined

cosmetics_data = list(cosmetics_joined_collection.find())
cosmetics_df = pd.DataFrame(cosmetics_data)

def calculate_rating(c_df, penalty_category):
    ingredients_ratings = []
    final_ratings = []

    for _, record in c_df.iterrows():
        ingredients_sum = 0
        ingredients_count = 0
        penalty_added = 1
        strong_penalty = ["Suspected endocrine disruptor", "Formaldehyde liberator", "Forbidden in Europe"]

        customer_rank = record["Rank"]
        cosmetic_ingredients = record["Ingredients"]

        for ingredient in cosmetic_ingredients:
            ingredient_details = ingredient.get("Details", None)

            if ingredient_details:
                penalty = ingredient_details.get(penalty_category, None)
                
                # not needed because if details don't exist it won't get to this part of code
                if penalty is None:
                    continue
                
                classification = ingredient_details.get("Classification", [])

                if penalty == "Strong":
                    ingredients_count += 1
                    if any(x in classification for x in strong_penalty):
                        penalty_added = 0
                    else:
                        penalty_added -= 0.5
                elif penalty == "Medium":
                    ingredients_count += 1
                    ingredients_sum += 5
                    penalty_added -= 0.04
                elif penalty == "Low":
                    ingredients_count += 1
                    ingredients_sum += 8
                    penalty_added -= 0.02
                elif penalty == "None":
                    ingredients_count += 1
                    ingredients_sum += 10

        if customer_rank == 0:
            rank_weight = 0
            ingredients_weight = 1
        else:
            customer_rank = (customer_rank - 1) * (10 / 4)
            rank_weight = 0.4
            ingredients_weight = 0.6

        if penalty_added < 0:
            penalty_added = 0
            
        if ingredients_count != 0:
            ingredients_avg = ingredients_sum / ingredients_count
            ingredients_rating = round(ingredients_avg * penalty_added, 2)
            final_rating = round((ingredients_rating * ingredients_weight) + (customer_rank * rank_weight), 2)
        else:
            ingredients_rating = -10
            final_rating = -10

        ingredients_ratings.append(ingredients_rating)
        final_ratings.append(final_rating)

    return ingredients_ratings, final_ratings

ingredients_ratings_col, final_ratings_col = calculate_rating(cosmetics_df, "Face penalty")

cosmetics_df["Ingredients_rating"] = ingredients_ratings_col
cosmetics_df["Final_rating"] = final_ratings_col

cosmetics_df.to_csv("cosmetics_rating_csv.csv", index=False)

In [8]:
for index, row in cosmetics_df.iterrows():
    cosmetic_id = row["_id"]
    inserted_value = row["Ingredients_rating"]

    cosmetics_joined_collection.update_one(
        {"_id": cosmetic_id},
        {"$set": {"Ingredients_rating": inserted_value}}  
    )

In [9]:
for index, row in cosmetics_df.iterrows():
    cosmetic_id = row["_id"]
    inserted_value = row["Final_rating"]

    cosmetics_joined_collection.update_one(
        {"_id": cosmetic_id},
        {"$set": {"Final_rating": inserted_value}} 
    )

In [10]:
cosmetics_joined_collection.update_many({}, {"$set": {"Status": "Current"}})

UpdateResult({'n': 956, 'nModified': 956, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

engineering_db = client.Engineering
cosmetics_joined_collection = engineering_db.CosmeticsJoined

for document in cosmetics_joined_collection.find():
    new_document = {
        "_id": document["_id"],
        "Label": document["Label"],
        "Brand": document["Brand"],
        "Name": document["Name"],
        "Price": document["Price"],
        "Rank": document["Rank"],
        "Ingredients": document["Ingredients"],
        "Skin": document["Skin"],
        "Ingredients_rating": document["Ingredients_rating"],
        "Final_rating": document["Final_rating"],
        "Date": document["Date"],
        "Status": document["Status"]
    }
    
    cosmetics_joined_collection.replace_one({"_id": document["_id"]}, new_document)

fields_to_check = ["Skin"]

def clean_empty_fields(collection, fields):
    for field in fields:
        collection.update_many(
            {field: {"$eq": None}},
            {"$unset": {field: ""}}
        )

clean_empty_fields(cosmetics_joined_collection, fields_to_check)
